

1.   ~~Распаковать архив~~
2.   ~~Отобрать по-леммно с перф и импф глаголами~~, только если и перф и импф кол-во больше 5. Заполнить леммы в соответствующий столбец, если нет. `Continue`, если нет контекста или словоформы.
3.   ~Выкинуть семпл, если словоформы нет в контексте~
4.   Собрать все контексты в единый словарь типа `IMPF: [...], PERF: [...]`
5.   Разметить тексты с пайморфи в словарях.
6.   Создать словарь соответствий разметки `pymorphy3` и НКРЯ
7.   Повторно пройтись по ДВГ-словоформам из столбца и заполнить грамматику, если нет, используя словарь.
8.   Сохранить заново в одноимённый `.json` в файле в папке `biaspectual_verbs_diploma`.
------------------
9.   Открыть все глаголы, бахнуть их в один датасет `pandas`
10.  Разметить тексты целиком, кроме ДВГ
11.  Преобразовать столбец с грамматикой в отдельные столбцы в соответствии с грамматическими катгориям. Чек прошлогодний ноутбук



In [ ]:
!pip install numpy==2.0.2 -q
!pip install pandas==2.2.2 -q
!pip install pymorphy3==2.0.3 -q
!pip install scikit-learn==1.6.1 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 27.4 MB/s eta 0:00:00


In [ ]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
import pymorphy3
import sklearn

from sklearn.model_selection import train_test_split

In [ ]:
!pip freeze

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unrar x -Y "/content/drive/MyDrive/biaspectual_verbs.rar" "/content/drive/MyDrive/Biaspectual_Verbs_Diploma/"

In [ ]:
dir = '/content/drive/MyDrive/Biaspectual_Verbs_Diploma/biaspectual_verbs'

def extract_true_bi_verbs(directory=dir):
  true_biaspectives = []
  dir_iterator = list(Path(directory).iterdir())
  labels = {'perf', 'impf'}
  for path in dir_iterator:
      if path.is_file() and path.suffix == '.json':
        lemma, label = path.name[:path.name.index('.json')].split('_')
        label = {label}
        f = lemma + '_' + next(iter(labels - label)) + '.json'
        nec_path = path.parent / f
        if not nec_path.exists():
          continue
        true_biaspectives.append(f)
  true_biaspectives.sort()
  return true_biaspectives

bi_verbs = extract_true_bi_verbs()
print(len(bi_verbs))

580




1.   Я прохожусь по множествам из словоформ. Возвращаю словарь **словоформа: разметка `pymorphy`**. НКРЯ-ую оставлять не надо - сложнее будет сравнивать на этапе rule-based модели
2.   Вид и время переписываются в соответствии с именем в `.json` файле. `if impf - pres, if perf - futr`. В остальных случаях вид расставляется в соответствии лишь с именем файла.
3.   Далее проход по всем словоформам в файле с разметкой. Доступ в словарь по `словоформе.lower()` и заполнение столбца `"грамматика"` через запятую с пробелом. Сортировка не нужна.



In [ ]:
m = pymorphy3.MorphAnalyzer()
m.parse('делает')[0].tag.aspect

'impf'

In [ ]:
# Available attributes are:
# POS, animacy, aspect, case, gender, involvement, mood, number, person, tense, transitivity and voice.

def mark_wordforms(wordforms, aspect, pymorph=m):
  tense_mapping = {'impf': 'pres', 'perf': 'futr'}
  wf_markup = {}
  for t in wordforms:
      if t:
          tp = pymorph.parse(t)[0]
          wf_markup[t] = {
              'POS': tp.tag.POS,
              '': '',
              'aspect': aspect,
              'tense': str(tense) if str(tense := tp.tag.tense) == 'past'
              else tense_mapping.get(aspect if tense else '', ''),
              'gender': str(tp.tag.gender) if tp.tag.gender else '',
              'mood': list(filter(bool,
                  (
                    str(mood) if (mood := tp.tag.mood) else '',
                    str(inv) if (inv := tp.tag.involvement) else ''
                  )
                )),
              'number': str(tp.tag.number) if tp.tag.number else '',
              'person': str(tp.tag.person) if tp.tag.person else '',
              'transitivity': str(tp.tag.transitivity) if tp.tag.transitivity else '',
              'voice': str(tp.tag.voice) if tp.tag.voice else ''
          }
  return wf_markup

In [ ]:
# создадим словарь соответствий между разметкой pymorphy и НКРЯ
grammar_cats = {
    'лицо': {'1per', '2per', '3per'},
    'число': {'sing', 'plur'},
    'вид': {'perf', 'impf'},
    'наклонение / форма': {'indc', 'impr,excl', 'impr,incl', 'INFN', 'GRND', 'PRTF'},
    'залог': {'actv', 'pssv'},
    'время': {'pres', 'futr', 'past'},
    'род': {'masc', 'femn', 'neut', 'ms-f'},
    'переходность': {'tran', 'intr'}
}

# не понадобился
rnc2pym = {
    '1-е лицо': '1per', '2-е лицо': '2per', '3-е лицо': '3per',
    'единственное': 'sing', 'множественное': 'plur',
    'совершенный': 'perf', 'несовершенный': 'impf',
    'изъявительное': 'indc', 'повелительное': 'impr,excl', 'повелительное 2': 'impr,incl', 'инфинитив': 'INFN', 'деепричастие': 'GRND', 'причастие': 'PRTF',
    'действительный': 'actv', 'страдательный': 'pssv', 'медиальный': '',
    'настоящее': 'pres', 'будущее': 'futr', 'прошедшее': 'past',
    'переходный': 'tran', 'непереходный': 'intr',
    'мужской': 'masc', 'женский': 'femn', 'средний': 'neut', 'общий': 'ms-f'
}

In [ ]:
def flatten(lst):
  return [i if isinstance(i, str) else ",".join(i) for i in lst]

In [ ]:
def preprocess_json(path):
  with open(path, 'r', encoding='utf-8') as f:
    data_unfiltered = json.load(f)
    data = list(filter(bool, data_unfiltered))
    f = pd.DataFrame(data)

  cols = list(f.columns) + ["грамматика_pm"]
  if not f.empty:
    if 'основной анализ' in f.columns:
      f = f.drop(columns=['основной анализ'])
    f = f.replace('', np.nan)
    f = f.dropna(subset=["словоформа", "контекст"])

    f = f[f.apply(lambda row: row["словоформа"] in row['контекст'], axis=1)]
    f["контекст"] = f["контекст"].str.replace("\n+", " ", regex=True)
    f = f[f.apply(lambda row: " " in row["контекст"], axis=1)]
    f = f.drop_duplicates(keep=False, subset="контекст")

    if not f.empty:
      lemma, aspect = path.name.split('/')[-1].split('_')
      aspect = aspect[:-5]
      f["лемма"] = f["лемма"].fillna(lemma)

      wf_marked_up = mark_wordforms(set(f['словоформа'].str.lower()), aspect, m)
      # по словоформе достанем словарь с разметкой
      f["грамматика_pm"] = f['словоформа'].apply(lambda row: list(filter(bool, mark.values() if isinstance((mark := wf_marked_up.get(row.lower(), '')), dict) and mark else [])))
      f["грамматика_pm"] = f["грамматика_pm"].apply(lambda row: flatten(row) if isinstance(row, list) else row)
      f["грамматика_pm"] = f["грамматика_pm"].apply(lambda row: ','.join([grammeme for grammeme in row if grammeme]))
      f["грамматика_pm"] = f["грамматика_pm"].str.replace(r',+', ',', regex=True)
      f.reset_index(drop=True)
      if not f.empty:
        f.to_json(path.parent.parent / 'biaspectual_verbs_preprocessed' / path.name, orient='records', force_ascii=False)
        return f
      else:
        return pd.DataFrame(columns=cols)
    else:
        return pd.DataFrame(columns=cols)
  else:
    del f
    return pd.DataFrame(columns=cols)

In [ ]:
def filter_verbs(directory=dir,
                 true_biaspectives=bi_verbs):
  verb_occs = pd.DataFrame()
  dir = Path(directory)
  for inx, f in enumerate(true_biaspectives, start=1):
    path = dir / f
    lemma_asp = f[:-5]
    try:
      if path.is_file() and path.suffix == '.json':
        if inx % 2:
          perf_preprocessed = preprocess_json(path)
        elif not inx % 2:
          impf_preprocessed = preprocess_json(path)
    except (AttributeError, TypeError) as e:
      print(e, lemma_asp)
  return

verbs_filtered = filter_verbs()

In [ ]:
directory = Path('/content/drive/MyDrive/Biaspectual_Verbs_Diploma/biaspectual_verbs_preprocessed')
bi_verbs_preprocessed = extract_true_bi_verbs(directory)
print(len(bi_verbs_preprocessed) // 2)

289


In [ ]:
def form_dataset(true_biaspectives=bi_verbs_preprocessed,
                 dir=directory,
                 balanced=True):
  data_formed = pd.DataFrame(columns=['основной анализ', 'лемма', 'словоформа',
                                      'контекст', 'грамматика_pm',
                                      'грамматика'])
  for inx in range(0, len(true_biaspectives), 2):
    path_impf = dir / true_biaspectives[inx]
    path_perf = dir / true_biaspectives[inx + 1]

    impf = pd.read_json(path_impf, orient='records')
    perf = pd.read_json(path_perf, orient='records')

    min_len = min((impf.shape[0], perf.shape[0],))
    impf = impf[:min_len] if balanced else impf
    perf = perf[:min_len] if balanced else perf
    data_formed = pd.concat([data_formed, impf, perf])

  data_formed = data_formed.drop_duplicates(keep=False, subset="контекст")
  data_formed = data_formed.reset_index(drop=True)
  data_formed['вид_совершенный'] = data_formed['грамматика_pm'].apply(lambda row: 'perf' in row)
  y = data_formed['вид_совершенный']
  data_formed = data_formed.drop(columns=['основной анализ', 'грамматика',
                                          'вид_совершенный'])
  stratify = None if balanced else y
  X_train, X_test, y_train, y_test = train_test_split(data_formed, y,
                                                      test_size=0.3,
                                                      random_state=666,
                                                      stratify=stratify
  )
  train = pd.merge(X_train, y_train, left_index=True, right_index=True).reset_index(drop=True)
  test = pd.merge(X_test, y_test, left_index=True, right_index=True).reset_index(drop=True)
  return train, test

In [ ]:
train_bal, test_bal = form_dataset(balanced=False)
train_unbal, test_unbal = form_dataset(balanced=True)

train_bal.to_json(directory.parent / 'balanced_2verbs_dataset' / 'train.json.zip', orient='records', force_ascii=False, compression='zip')
test_bal.to_json(directory.parent / 'balanced_2verbs_dataset' / 'test.json.zip', orient='records', force_ascii=False, compression='zip')

train_unbal.to_json(directory.parent / 'unbalanced_2verbs_dataset' / 'train.json.zip', orient='records', force_ascii=False, compression='zip')
test_unbal.to_json(directory.parent / 'unbalanced_2verbs_dataset' / 'test.json.zip', orient='records', force_ascii=False, compression='zip')

In [ ]:
train_bal.shape, test_bal.shape

((11886, 5), (5095, 5))

In [ ]:
train_bal['вид_совершенный'].value_counts()

,count
вид_совершенный,
True,5952
False,5934


In [ ]:
test_bal['вид_совершенный'].value_counts()

,count
вид_совершенный,
False,2554
True,2541


In [ ]:
train_unbal.shape, test_unbal.shape

((31745, 5), (13605, 5))

In [ ]:
train_unbal['вид_совершенный'].value_counts()

,count
вид_совершенный,
False,22807
True,8938


In [ ]:
test_unbal['вид_совершенный'].value_counts()

,count
вид_совершенный,
False,9864
True,3741


In [ ]:
!unzip "/content/drive/MyDrive/Biaspectual_Verbs_Diploma/balanced_2verbs_dataset/test.json.zip"

unzip:  cannot find or open /content/drive/MyDrive/Biaspectual_Verbs_Diploma/balanced_2verbs_dataset/test.zip, /content/drive/MyDrive/Biaspectual_Verbs_Diploma/balanced_2verbs_dataset/test.zip.zip or /content/drive/MyDrive/Biaspectual_Verbs_Diploma/balanced_2verbs_dataset/test.zip.ZIP.


In [ ]:
df = pd.read_json("/content/test", orient='records')
df

<ipython-input-25-c4981a61993e>:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json("/content/test", orient='records')


ValueError: Expected object or value